# Model Analysis

#### Model Performance Accuracy
    Linear Regression: 22.89%
    Logistic Regression: 89.93%
    SVM: 93.41%  KAGGLE: 91.22%
    NN: 96.25%  KAGGLE: 
    
    KNN: Too many features for my device

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
df_train = pd.read_csv(path+'train_data.csv')
df_val = pd.read_csv(path+'val_data.csv')
df_test = pd.read_csv(path+'test_data.csv')

In [2]:
# Converting dataframes -> numpy arrays to train models off of 
train = df_train.values
val = df_val.values
test = df_test.values

Xtr = train[:,1:]
Ytr = train[:,0]
Xval = val[:,1:]
Yval = val[:,0]
Xte = test[:,1:]
Yte = test[:,0]

In [3]:
# Normalizing data to use simple Keras models
from sklearn import preprocessing
Xtr = preprocessing.normalize(Xtr)
Xval = preprocessing.normalize(Xval)
Xte = preprocessing.normalize(Xte)

In [4]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score

regr = linear_model.LinearRegression()
regr.fit(Xtr, Ytr)
Yhat = np.floor(regr.predict(Xval))
print("Linear model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


Linear model accuracy: 24.44%


In [5]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(Xtr, Ytr)
Yhat = lr.predict(Xval)
print("Linear model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

Linear model accuracy: 89.93%


In [6]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier

Yhat = OneVsOneClassifier(LinearSVC(random_state=0)).fit(Xtr, Ytr).predict(Xval)
print("SVM model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

SVM model accuracy: 93.41%


In [7]:
# Takes much longer than SVM
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()
clf.fit(Xtr, Ytr)
Yhat = clf.predict(Xval)
print("NN model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

NN model accuracy: 96.17%


KNN takes too long on my device due to all the features present in this problem

In [8]:
path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
test_model = pd.read_csv(path+'test.csv')

In [15]:
path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
train_model = pd.read_csv(path+'model_train.csv')

In [16]:
# Removing all features from test.csv not evaluated in our model
for i in range(784):
    col = "pixel" + str(i)
    if col not in train_model.columns:
        test_model = test_model.drop([col], axis=1)

KeyError: "labels ['pixel0'] not contained in axis"

In [37]:
Yhat = OneVsOneClassifier(LinearSVC(random_state=0)).fit(train_model.iloc[:,1:], train_model.iloc[:,0]).predict(test_model)

In [60]:
# Creating submission file
index = np.arange(1, Yhat.shape[0]+1)
submission = np.column_stack((index,Yhat))
np.savetxt('submission.txt', submission, delimiter=',', fmt='%i') # I add: "ImageId,Label" manually to first row

*Train model on raw train set, and on non uniform set, and on pixels included